# Cuaderno 20: Problema de enrutamiento vehicular capacitado (CVRP)

Dados
* Un grafo dirigido **completo** $D = (V,A)$, donde el conjunto de nodos $V = N \cup \{0\}$ se compone de un subconjunto $N := \{1, \ldots, n\}$ de nodos que representan *clientes* y un nodo $0$ que representa un *depósito*;
* un número $K \in \mathbb{N}$ que representa el número de vehículos de una flota (todos los vehículos son idénticos entre sí);
* un número $Q \in \mathbb{N}$ que representa la capacidad de cada vehículo;
* un vector de costos $c \in \mathbb{Z}^{A}_{+}$ asociados a los arcos del grafo $D$; y,
* un vector de demandas  $d \in \mathbb{Z}^{N}_{+}$ asociados a los nodos $N$.

El *problema de enrutamiento vehicular capacitado* (CVRP, *Capacitated Vehicle Routing Problem*) consiste en diseñar un conjunto de rutas para atender a los clientes, empleando una flota de $K$ vehículos idénticos entre sí, bajo las siguientes condiciones:

* Cada ruta debe empezar y terminar en el nodo depósito $0$, es decir, cada ruta es un circuito dirigido en $D$ que contiene al nodo $0$;
* el número de rutas a utilizar **no puede exceder** el número $K$ de vehículos disponibles;
* cada cliente debe ser visitado exactamente una vez, dentro de alguna ruta;
* la suma de las demandas de los clientes visitados en una misma ruta no puede exceder la capacidad $Q$ del vehículo (asumimos que $d_i \leq Q$ para todo $i \in N$, pues de lo contrario el problema no tiene solución factible); y
* debe minimizarse el costo total de todas las rutas, donde el costo de una ruta es la suma de los costos de los arcos que la componen.

Para formular este problema como un programa lineal entero, emplearemos las siguientes variables de decisión:

   * Variables binarias de selección de arcos $x_{ij}$, con $(i, j) \in A$, tales que $x_{ij} = 1$ si y solamente si el arco $(i,j)$ es usado en alguna ruta.
   * Variables no negativas $L_i$, con $0 \leq L_i \leq Q, \forall i \in V$, que indican la *carga* del vehículo después de visitar un nodo $i$.
   
Con estas variables, el modelo puede expresarse en la siguiente forma:

\begin{align*}
\min &\sum_{(i,j) \in A} c_{ij} x_{ij}\\ 
& \text{s.r.}\\
&\sum_{(i,j) \in A} x_{ij} = 1, \quad \forall i \in N,\\
&\sum_{(j,i) \in A} x_{ij} = 1, \quad \forall i \in N,\\
&\sum_{(0,i) \in A} x_{0i} \leq K,\\
&\sum_{(i,0) \in A} x_{i0} \leq K,\\
&L_j \geq L_i + d_{j} - M(1 -  x_{ij}), \quad \forall (i,j) \in A, j \neq 0,\\
&L_0 = 0, \\
&0 \leq L_i \leq Q, \quad \forall i \in V,\\ 
&  x_{ij} \in \{0, 1\}, \quad \forall (i,j) \in A.\\
\end{align*}

La función objetivo mide el costo total de todos los arcos seleccionados en las rutas.

Las dos primeras familias de restricciones son restricciones de grado para los nodos clientes. Especifican que cada cliente $i \in N$ debe ser visitado exactamente por una ruta, pues deben seleccionarse un arco entrante a $i$ y un arco saliente de $i$.

La tercera y cuarta restricciones son restricciones de grado para el depósito. Indican que la solución debe consistir de máximo $K$ rutas.

La quinta familia de restricciones fija los valores de la variable de carga $L_j$ a lo largo de una ruta, para que sean mayores o iguales a las sumas de las demandas de los clientes atendidos por la ruta. Notar que esta restricción es del tipo *$M$-mayúscula*, y que basta seleccionar $M = 2Q$.

La sexta restricción indica que los vehículos salen del depósito sin carga, es decir, con toda su capacidad disponible.

Finalmente, la séptima familia de restricciones (cotas inferiores y superiores sobre las variables $L_i$) especifican que la carga de un vehículo nunca debe superar su capacidad.

Vamos a implementar el modelo usando la interfaz Python de Gurobi, sobre la instancia indicada a continuación:

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import random as rm
import math

# semilla para el generador de números aleatorios
rm.seed(0)

# número de nodos clientes
n = 10

# número de vehículos disponibles
K = 4

# conjunto de clientes N
N = gp.tuplelist(i+1 for i in range(n))

# conjunto de nodos V 
V = gp.tuplelist(N + [0])

# demandas de nodos clientes 
d = gp.tupledict({i : rm.randint(3,10) for i in N})

# coordenadas de los nodos 
coordx={i : rm.randint(0,100) for i in N}
coordy={i : rm.randint(0,100) for i in N}
# depósito está en el centro
coordx[0]= 50
coordy[0]= 50

# los costos son las distancias eculideanas
c = gp.tupledict({
    (i,j) : math.sqrt((coordx[i] - coordx[j])**2 + (coordy[i] - coordy[j])**2)
    for i in V for j in V if i!=j
})
 
A = c.keys()
            
# capacidad de los vehículos
Q = 25

# definir constante M 
M = 2*Q
            


Creamos primero el objeto modelo, las variables y la función objetivo:

In [ ]:
### crear el objeto modelo #### 
m = gp.Model('CVRP')

#### variables de decisión #### 
# variables de selección de cargo
x = m.addVars(A, name="x", vtype=GRB.BINARY)
# variables de carga despues de la visita a nodos
L = m.addVars(V, name="L",lb=0, ub=Q)

#### función objetivo #### 
m.setObjective(x.prod(c,'*'), GRB.MINIMIZE)

Agregamos ahora las restricciones de grado de los nodos clientes:

In [ ]:
# restricciones de grado saliente nodos clientes
m.addConstrs((x.sum(i,'*') == 1 for i in N), "g_saliente")
# restricciones de grado entrante nodos clientes
m.addConstrs((x.sum('*', i) == 1 for i in N), "g_entrante")

Las restricciones de grado en el nodo depósito limitan la cantidad de rutas a utilizar en la solución:

In [ ]:
# restricción de grado saliente del nodo depósito 
m.addConstr(x.sum(0,'*')  <= K, "deposito_saliente")
# restricción de grado entrante del nodo depósito 
m.addConstr(x.sum('*',0)  <= K, "deposito_entrante")

Por último, añadimos las restricciones de carga de los vehículos:

In [ ]:
# restricciones de carga en los nodos intermedios de las rutas
m.addConstrs((L[j] >= L[i] + d[j] - M*(1 - x[i, j]) for (i,j) in A if j!=0), "carga_intermedia")
# restricción de carga al salir del depósito
m.addConstr((L[0] == 0), "carga_inicial")

Una vez definido el modelo, llamamos a `optimize` para resolverlo, y en el caso de que sea posible encontrar al menos una solución factible, mostramos la mejor solución encontrada:

In [ ]:
# solución del modelo
m.optimize()

# escritura de la mejor respuesta, si se encontró al menos una solución factible:
if m.SolCount > 0:
    # recuperar los valores de las variables
    vx = m.getAttr('x', x)    
    vL = m.getAttr('x', L)
    print('Rutas:')
    for i,j in A:
        if vx[i,j] >= 0.99:
            print('{0} -> {1}'.format(i, j))
    print('Carga visita:')
    for i in N:
        print('L[{}]= {}'.format(i, int(vL[i])))
    print("Costo: {}".format(m.objval))

Al igual que en el caso del $k$-ATSP, es más informativo mostrar la secuencia de nodos visitados por cada ruta:

In [ ]:
# crear lista con arcos seleccionados en la solución
vx = m.getAttr('x', x)
L = gp.tuplelist([(i,j) for (i,j) in A if vx[i,j]>=0.9])

# recuperar el número de rutas
n_rutas = len(L.select(0,'*'))

# recuperar cada ruta como un ordenamiento de los nodos que empieza y termina en el depósito
rutas = []
for i in range(n_rutas):
    T = [0]
    # nodo actual:
    i = 0
    while True:
        # determinar un sucesor de i
        (i,j) = L.select(i,'*').pop(0)
        T.append(j)
        i = j
        if i==0:
            break
    rutas.append(T)

print ('*** Rutas óptimas (con cargas por ruta):')
for i in range(len(rutas)):
    dT = sum(d[j] for j in rutas[i] if j!=0)
    print('r{0}: {1}; dT{0}={2}'.format(i+1, rutas[i], dT))


También es posible graficar la solución empleando la biblioteca `matplotlib`:

In [ ]:
import matplotlib.pyplot as plt
import random

def dibujarRutas(coordx, coordy, rutas, d):
    plt.figure(figsize=(10,10))
    for tour in rutas:
        Tx = [coordx[i] for i in tour]
        Ty = [coordy[i] for i in tour]
        plt.plot(Tx[:-1], Ty[:-1], 'ro')
        for i in range(len(tour)-1):
            s='{}'.format(tour[i])
            plt.text(Tx[i],Ty[i]+1,s)
            if tour[i]>0:
                s2='{}'.format(d[tour[i]])
                plt.text(Tx[i]-1,Ty[i]-3,s2, color='green')
            plt.arrow(Tx[i], Ty[i], Tx[i+1]-Tx[i], Ty[i+1]-Ty[i], color='blue', 
                  length_includes_head=True, width=0.1, head_width=1)
    plt.show()

# Graficar el tour
dibujarRutas(coordx, coordy, rutas, d) 

## Código completo

Se reproduce a continuación el código completo del modelo anterior.

In [ ]:
# Implementación de modelos lineales enteros
# Problema de enrutamiento vehicular capacitado (CVRP)

# Luis M. Torres (EPN 2023)

import gurobipy as gp
from gurobipy import GRB
import random as rm
import math
import matplotlib.pyplot as plt
import math

def dibujarRutas(coordx, coordy, rutas, d):
    plt.figure(figsize=(10,10))
    for tour in rutas:
        Tx = [coordx[i] for i in tour]
        Ty = [coordy[i] for i in tour]
        plt.plot(Tx[:-1], Ty[:-1], 'ro')
        for i in range(len(tour)-1):
            s='{}'.format(tour[i])
            plt.text(Tx[i],Ty[i]+1,s)
            if tour[i]>0:
                s2='{}'.format(d[tour[i]])
                plt.text(Tx[i]-1,Ty[i]-3,s2, color='green')
            plt.arrow(Tx[i], Ty[i], Tx[i+1]-Tx[i], Ty[i+1]-Ty[i], color='blue', 
                  length_includes_head=True, width=0.1, head_width=1)
    plt.show()

# semilla para el generador de números aleatorios
rm.seed(0)

# número de nodos clientes
n = 30

# número de vehículos disponibles
K = 3

# conjunto de clientes N
N = gp.tuplelist(i+1 for i in range(n))

# conjunto de nodos V 
V = gp.tuplelist(N + [0])

# demandas de nodos clientes 
d = gp.tupledict({i : rm.randint(10,50) for i in N})

# coordenadas de los nodos 
coordx={i : rm.randint(0,100) for i in N}
coordy={i : rm.randint(0,100) for i in N}
# depósito está en el centro
coordx[0]= 50
coordy[0]= 50

# los costos son las distancias eculideanas
c = gp.tupledict({
    (i,j) : math.sqrt((coordx[i] - coordx[j])**2 + (coordy[i] - coordy[j])**2)
    for i in V for j in V if i!=j
})
 
A = c.keys()
            
# capacidad de los vehículos
Q = 320

# definir constante M 
M = 2*Q

try:
    # crear el objeto modelo
    m = gp.Model('CVRP')

    # variables de selección de arcos
    x = m.addVars(A, name="x", vtype=GRB.BINARY)
    # variables de carga despues de la visita a nodos
    L = m.addVars(V, name="L",lb=0, ub=Q)

    # función objetivo 
    m.setObjective(x.prod(c,'*'), GRB.MINIMIZE)

    ### restricciones ###

    # restricciones de grado saliente clientes
    m.addConstrs((x.sum(i,'*') == 1 for i in N), "g_saliente")
    # restricciones de grado entrante clientes
    m.addConstrs((x.sum('*', i) == 1 for i in N), "g_entrante")

    # restricción de grado saliente del nodo depósito 
    m.addConstr(x.sum(0,'*')  <= K, "deposito_saliente")
    # restricción de grado entrante del nodo depósito 
    m.addConstr(x.sum('*',0)  <= K, "deposito_entrante")

    # restricciones de carga en los nodos intermedios de las rutas
    m.addConstrs((L[j] >= L[i] + d[j] - M*(1 - x[i, j]) for (i,j) in A if j!=0), "carga_intermedia")
    # restricción de carga al salir del depósito
    m.addConstr((L[0] == 0), "carga_inicial")

    # fijar el tiempo límite de cálculo en 180 segundos
    m.Params.TimeLimit = 180
    
    # fijar brecha de optimalidad aceptable en 20%
    m.Params.MIPGap = 0.2
    
    # resolver el modelo 
    m.optimize()
    
    ### si se encontró alguna solución factible, mostrar la mejor solución
    if m.SolCount > 0:
    
        # crear lista con arcos seleccionados en la solución
        vx = m.getAttr('x', x)
        L = gp.tuplelist([(i,j) for (i,j) in A if vx[i,j]>=0.9])

        # recuperar el número de rutas
        n_rutas = len(L.select(0,'*'))

        # recuperar cada ruta como un ordenamiento de los nodos que empieza y termina en el depósito
        rutas = []
        for i in range(n_rutas):
            T = [0]
            # nodo actual:
            i = 0
            while True:
                # determinar un sucesor de i
                (i,j) = L.select(i,'*').pop(0)
                T.append(j)
                i = j
                if i==0:
                    break
            rutas.append(T)

        print ('*** Rutas óptimas:')
        for i in range(len(rutas)):
            dT = sum(d[j] for j in rutas[i] if j!=0)
            print('r{0}: {1}; dT{0}={2}'.format(i+1, rutas[i], dT))
            
        # dibujar las rutas
        dibujarRutas(coordx, coordy, rutas, d) 

except GurobiError as e:
    print('Se produjo un error de Gurobi: código: ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Se produjo un error de atributo')